In [1]:
import numpy as np
import xarray as xr
from datetime import datetime as dt
import sys

In [2]:
# ************************************************************************************************
# *************************** USER INPUTS ARE REQUIRED IN THIS CELL ******************************
# ************************************************************************************************


# Setup strings that will be concatenated together to form the path to the file

# List all the variables to be uploaded
varName = ['PRECT', 'VBOT', 'UBOT']

# List all the hindcast days to be loaded
hcastDayStr = [None,'01','02','03','04','05','06','07','08','09','10','11','12','13','14','15','16','17','18','19','20']
hcastDay = [None,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]

# These are the base strings of the hindcast file paths/names
fn = 'tratl_cam5.3_ne30_rneale.globe.fcast.day'
dri = '/home/jlarson1/data/CAM_hindcasts/'    

## Observation precipitation
# driObs = '/lss/research/agon-lab/TMPA-daily/'
driPRECTObs = '/home/jlarson1/data/'
fnPRECTObs  = 'pr_daily_2009-2010.nc'

## Observation surface winds
# Note that these two scripts are assuming a single dataset each for vbot and ubot
# If the hindcast data spans multiple years, the user is advised to concatenate the years together to avoid modying the scripts
driWindObs = '/home/jlarson1/data/ERA5/'
fnUBOTObs = 'u10m_2009-2010-partial_daily.nc'
fnVBOTObs = 'v10m_2009-2010-partial_daily.nc'


# ************************************************************************************************
# ************************************************************************************************
# ************************************************************************************************

In [3]:
# Assemble all the loaded data into two datasets: hindcasts and obs

numVars = len(varName)

# Initialize empty array to hold all the data
megaDataImport = [[None] * len(hcastDayStr), [None] * len(hcastDayStr), [None] * len(hcastDayStr)]
dataMegaObs = [None] * numVars

# Loop through each variables
for ii, _var in enumerate(varName):
    # Loop through each hindcast day
    for jj, day in enumerate(hcastDayStr):
        if jj == 0: continue # Skip zero index so that the hindcast indexing is more intuitive (e.g. day 7 hindcast is indexed with a 7)
            
#         print(dri+fn+day+'.'+_var+'.nc')
        megaDataImport[ii][jj] = xr.open_dataset(dri+fn+day+'.'+_var+'.nc')
    
# prect
dataMegaObs[0] = xr.open_dataset(driPRECTObs + fnPRECTObs)['pr']


# vbot. Flip latitude around so it matches hcast convention
# dataMegaObs[1] = xr.open_dataset(driWindObs + fnVBOTObs)
dataVbotObs = xr.open_dataset(driWindObs + fnVBOTObs)['v10']
dataMegaObs[1] = xr.DataArray(dataVbotObs[:,::-1,:], dims=('time','lat','lon'), 
                              coords={'time':dataVbotObs.time.values, 'lat':dataVbotObs.latitude.values[::-1], 'lon':dataVbotObs.longitude.values})

# ubot
# dataMegaObs[2] = xr.open_dataset(driWindObs + fnUBOTObs)
dataUbotObs = xr.open_dataset(driWindObs + fnUBOTObs)['u10']
dataMegaObs[2] = xr.DataArray(dataUbotObs[:,::-1,:], dims=('time','lat','lon'), 
                              coords={'time':dataUbotObs.time.values, 'lat':dataUbotObs.latitude.values[::-1], 'lon':dataUbotObs.longitude.values})

In [5]:
# Convert time stamps from cftime to datetime format

# Initialize array to hold every day's (152 days) time stamp for each hindcast day (20 days)
prectTime = np.array([ [None] * len(megaDataImport[0][1].time) ] * len(megaDataImport[0]))

# Loop through all hindcast days
# Start the index at 1 so the indexing is more intuitive (e.g. day 7 hindcast is indexed with a 7)
for ii in range(1,  len(megaDataImport[0]) ):
    
    # Loop through each day in the hindcast dataset
    for jj in range( len(megaDataImport[0][1].time) ):
        # Parse through the time stamp and convert to the datetime class
        # The index inside the str below is as follows: variable (precip)->hcastDay->time dim->specific time stamp->raw value
        prectTime[ii][jj] = dt.strptime(str(megaDataImport[0][ii]['time'][jj].values), '%Y-%m-%d %H:%M:%S')

In [6]:
# Time average the 3-hourly wind hindcast data to get daily wind data

# Create an empty array to hold the daily averaged data
vbot = np.zeros([len(hcastDayStr), len(megaDataImport[1][1].time)//8, len(megaDataImport[1][1].lat), len(megaDataImport[1][1].lon)])
ubot = np.zeros([len(hcastDayStr), len(megaDataImport[2][1].time)//8, len(megaDataImport[2][1].lat), len(megaDataImport[2][1].lon)])

# Iterate for the number of hindcast days
for ii in range(1, len(vbot) ):

    # Iterate for the number of time stamps in 3hrly data. Count by 8
    for jj in range(0, len(vbot[1])*8, 8):
        # On the left, jj is divided by 8 so it increases by 1 (each day)
        # On the right, 8 is added to jj to include all 8 daily data points in the mean
        vbot[ii][jj//8] = megaDataImport[1][ii]['V'][jj:jj+8].mean(dim='time').values
        ubot[ii][jj//8] = megaDataImport[2][ii]['U'][jj:jj+8].mean(dim='time').values

In [7]:
# Recreate the mega dataset so that it has the newly fixed time stamps for all variables
# and so that the horizontal winds are daily averaged

# Initialize an empty array to hold the new dataArrays
dataMega = [[None] * len(hcastDayStr), [None] * len(hcastDayStr), [None] * len(hcastDayStr)]


# Loop through each hindcast day. Skip the zero (empty) index
for ii in range(1, len(dataMega[1]) ):

    # Convert precip from m/s to mm/day
    dataMega[0][ii] = xr.DataArray(megaDataImport[0][ii].PRECT*1000*86400, dims=['time','lat','lon'],
                                   coords={'time':prectTime[ii], 'lat':megaDataImport[0][1].lat, 'lon':megaDataImport[0][1].lon})
    
    dataMega[1][ii] = xr.DataArray(vbot[ii], dims=['time','lat','lon'],
                                   coords={'time':prectTime[ii], 'lat':megaDataImport[1][1].lat, 'lon':megaDataImport[1][1].lon})
    
    dataMega[2][ii] = xr.DataArray(ubot[ii], dims=['time','lat','lon'],
                                   coords={'time':prectTime[ii], 'lat':megaDataImport[2][1].lat, 'lon':megaDataImport[2][1].lon})
